# D. State stabilization in a noisy environment with Jaynes-Cummings controls

Here the result is as follows: The algorithm optimizes the params, such that the POVM always outputs 1, implying that the measurement leaves the target state invariant. this is what we are indeed seeing when printing the measurement outcome and its probability, when batching however, the optimizer struggles to converge.

Also, lookup here is much better than nn for the same hyperparameters.

This is actually a perfect example, of feedback grape modifying the params so that a certain measurement sequence will always be output because this measurement sequence is the one that is going to lead to the best fidelity

In [92]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [93]:
from feedback_grape.fgrape import optimize_pulse
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

In [94]:
N_cav = 30

## Here, dividing alpha into real and imaginary parts complicates the optimization and converges at 0.89 while if we do not use the imaginary part it converges at 0.999

In [95]:
def qubit_unitary(alpha_re):
    alpha = alpha_re[0]
    return tensor(
        identity(N_cav),
        expm(-1j * (alpha * sigmap() + alpha.conjugate() * sigmam()) / 2),
    )

In [96]:
def qubit_cavity_unitary(beta_re):
    beta = beta_re[0]
    return expm(
        -1j
        * (
            beta * (tensor(destroy(N_cav), sigmap()))
            + beta.conjugate() * (tensor(create(N_cav), sigmam()))
        )
        / 2
    )

### povm_measure_operator (callable): <br>
    - It should take a measurement outcome and list of params as input
    - The measurement outcome options are either 1 or -1

In [97]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, params):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    gamma, delta = params
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2 * identity(2*N_cav)
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

In [98]:
from feedback_grape.utils.states import coherent

alpha = 3
psi_target = tensor(
    coherent(N_cav, alpha)
    + coherent(N_cav, -alpha)
    + coherent(N_cav, 1j * alpha)
    + coherent(N_cav, -1j * alpha),
    basis(2),
)  # 4-legged state

# Normalize psi_target before constructing rho_target
psi_target = psi_target / jnp.linalg.norm(psi_target)
rho_target = psi_target @ psi_target.conj().T

In [99]:
rho_target.shape

(60, 60)

### It is important to test what the POVM probability is, to check if your state is normalized. if the probability is bounded between 0 and 1 then normalized
This completeness condition on the measurement operators is automatically checked for the `initial_params` in `optimize_pulse`.

In [100]:
# Answer: should one normalize within the optimization just in case?
# --> no so that user is not misled into thinking that his unnormalized state
# is working properly
from feedback_grape.utils.povm import (
    _probability_of_a_measurement_outcome_given_a_certain_state,
)

_probability_of_a_measurement_outcome_given_a_certain_state(
    rho_target,
    +1,
    povm_measure_operator(+1, params=[0.058, jnp.pi / 2]),
    povm_measure_operator(-1, params=[0.058, jnp.pi / 2]),
    evo_type="density",
)

Array(0.09369074, dtype=float64)

In [101]:
from feedback_grape.utils.fidelity import fidelity

print(fidelity(U_final=rho_target, C_target=rho_target, evo_type="density"))

1.0000000093148405


### Without dissipation

In [104]:
# Here the loss directly corressponds to the -fidelity (when converging) because log(1) is 0 and
# the algorithm is choosing params that makes the POVM generate prob = 1
from feedback_grape.fgrape import Gate, Decay

measure = Gate(
    gate=povm_measure_operator,
    initial_params=[0.058, jnp.pi / 2],  # gamma and
    measurement_flag=True,
)

qub_unitary = Gate(
    gate=qubit_unitary,
    initial_params=[jnp.pi / 3],  # alpha
    measurement_flag=False,
    # "param_constraints": [[0, 0.5], [-1, 1]],
)

qub_cav = Gate(
    gate=qubit_cavity_unitary,
    initial_params=[jnp.pi / 3],  # beta
    measurement_flag=False,
    # "param_constraints": [[0, 0.5], [-1, 1]],
)


system_params = [measure, qub_unitary, qub_cav]
for reward_weights in [(1,1),(0,1)]:
    result = optimize_pulse(
        U_0=rho_target,
        C_target=rho_target,
        system_params=system_params,
        num_time_steps=2,
        reward_weights=reward_weights,
        mode="lookup",
        goal="fidelity",
        max_iter=1000,
        convergence_threshold=1e-16,
        learning_rate=0.005,
        evo_type="density",
        batch_size=1,
        eval_time_steps=5,
        progress=True
    )

    print(f"reward weights: {reward_weights}\n final_fidelity: {result.final_fidelity}\n fidelity_each_timestep: {jnp.array(result.fidelity_each_timestep)}\n")

Iteration 10, Loss: -0.434777, T=0s, eta=51s
Iteration 20, Loss: -0.469147, T=1s, eta=55s
Iteration 30, Loss: -0.430738, T=1s, eta=53s
Iteration 40, Loss: -0.414021, T=2s, eta=51s
Iteration 50, Loss: -0.425890, T=2s, eta=51s
Iteration 60, Loss: -0.405357, T=3s, eta=50s
Iteration 70, Loss: -0.408244, T=3s, eta=50s
Iteration 80, Loss: -0.441015, T=4s, eta=51s
Iteration 90, Loss: -0.460492, T=5s, eta=51s
Iteration 100, Loss: 0.017883, T=5s, eta=50s
Iteration 110, Loss: -0.471190, T=6s, eta=49s
Iteration 120, Loss: -0.475588, T=6s, eta=48s
Iteration 130, Loss: -0.479252, T=7s, eta=47s
Iteration 140, Loss: -0.482890, T=7s, eta=46s
Iteration 150, Loss: -0.482492, T=8s, eta=45s
Iteration 160, Loss: -0.487008, T=8s, eta=44s
Iteration 170, Loss: -0.491061, T=9s, eta=43s
Iteration 180, Loss: -0.465822, T=9s, eta=43s
Iteration 190, Loss: -0.312555, T=10s, eta=42s
Iteration 200, Loss: -0.432598, T=10s, eta=42s
Iteration 210, Loss: -0.496051, T=11s, eta=41s
Iteration 220, Loss: -0.493484, T=11s, et

In [106]:
result.optimized_trainable_parameters["initial_params"]

[Array([0.03819767, 2.39149717], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]

In [107]:
result.returned_params

[[Array([[0.03819767, 2.39149717],
         [0.03819767, 2.39149717],
         [0.03819767, 2.39149717],
         [0.03819767, 2.39149717],
         [0.03819767, 2.39149717],
         [0.03819767, 2.39149717],
         [0.03819767, 2.39149717],
         [0.03819767, 2.39149717],
         [0.03819767, 2.39149717],
         [0.03819767, 2.39149717]], dtype=float64),
  Array([[-2.78661739e-06],
         [-2.78661739e-06],
         [-2.78661739e-06],
         [-2.78661739e-06],
         [-2.78661739e-06],
         [-2.78661739e-06],
         [-2.78661739e-06],
         [-2.78661739e-06],
         [-2.78661739e-06],
         [-2.78661739e-06]], dtype=float64),
  Array([[0.94894431],
         [0.94894431],
         [0.94894431],
         [0.94894431],
         [0.94894431],
         [0.94894431],
         [0.94894431],
         [0.94894431],
         [0.94894431],
         [0.94894431]], dtype=float64)],
 [Array([[0.05792869, 1.90432359],
         [0.05792869, 1.90432359],
         [0.057928

In [108]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.0000000093148405
fidelity of state 0: 0.004827095072388107
fidelity of state 1: 0.004827095072388107
fidelity of state 2: 0.004827095072388107
fidelity of state 3: 0.004827095072388107
fidelity of state 4: 0.004827095072388107
fidelity of state 5: 0.004827095072388107
fidelity of state 6: 0.004827095072388107
fidelity of state 7: 0.004827095072388107
fidelity of state 8: 0.004827095072388107
fidelity of state 9: 0.0028830001872770955


### With Dissipation

In [109]:
# Note if tsave = jnp.linspace(0, 1, 1) = [0.0] then the decay is not applied ?
# because the first time step has the original non decayed state
# Question: Here I should not use any sort of Hamiltonian or tspan or something?

decay = Decay(
    c_ops=[tensor(identity(N_cav), jnp.sqrt(0.10) * sigmam())],
)
system_params = [decay, measure, qub_unitary, qub_cav]
for reward_weights in [(1,1),(0,1)]:
    result = optimize_pulse(
        U_0=rho_target,
        C_target=rho_target,
        system_params=system_params,
        num_time_steps=2,
        reward_weights=reward_weights,
        mode="lookup",
        goal="fidelity",
        max_iter=1000,
        convergence_threshold=1e-6,
        learning_rate=0.005,
        evo_type="density",
        batch_size=1,
        eval_time_steps=5,
        progress=True
    )

    print(f"reward weights: {reward_weights}\n final_fidelity: {result.final_fidelity}\n fidelity_each_timestep: {jnp.array(result.fidelity_each_timestep)}\n")

Iteration 10, Loss: -0.398303, T=0s, eta=79s
Iteration 20, Loss: -0.435848, T=1s, eta=82s
Iteration 30, Loss: -0.442211, T=2s, eta=83s
Iteration 40, Loss: -0.426085, T=3s, eta=83s
Iteration 50, Loss: -0.388271, T=4s, eta=82s
Iteration 60, Loss: 0.117361, T=5s, eta=81s
Iteration 70, Loss: -0.227680, T=6s, eta=80s
Iteration 80, Loss: -0.228944, T=7s, eta=80s
Iteration 90, Loss: -0.233368, T=7s, eta=79s
Iteration 100, Loss: -0.231770, T=8s, eta=78s
Iteration 110, Loss: -0.233944, T=9s, eta=77s
Iteration 120, Loss: 0.131030, T=10s, eta=76s
Iteration 130, Loss: -0.231770, T=11s, eta=76s
Iteration 140, Loss: -0.228728, T=12s, eta=75s
Iteration 150, Loss: -0.206972, T=13s, eta=74s
Iteration 160, Loss: 0.104981, T=14s, eta=73s
Iteration 170, Loss: 0.072543, T=14s, eta=72s
Iteration 180, Loss: 0.079497, T=15s, eta=71s
Iteration 190, Loss: 0.124889, T=16s, eta=71s
Iteration 200, Loss: -0.245274, T=17s, eta=70s
Iteration 210, Loss: -0.247281, T=18s, eta=69s
Iteration 220, Loss: -0.248583, T=19s, 

In [110]:
print(result.final_fidelity)
print(jnp.array(result.fidelity_each_timestep))

0.09989977847322556
[1.00000001e+00 7.97559054e-04 6.78304485e-01 1.05128159e-02
 1.42811629e-01 9.98997785e-02]


In [111]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.0000000093148405
fidelity of state 0: 0.001521807302458907
fidelity of state 1: 0.137835456053904
fidelity of state 2: 0.019578787673703902
fidelity of state 3: 0.137835456053904
fidelity of state 4: 0.137835456053904
fidelity of state 5: 0.137835456053904
fidelity of state 6: 0.137835456053904
fidelity of state 7: 0.137835456053904
fidelity of state 8: 0.013048997378764634
fidelity of state 9: 0.137835456053904


In [112]:
result.returned_params

[[Array([[0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838],
         [0.0376334 , 2.39761838]], dtype=float64),
  Array([[-5.88306427e-05],
         [-5.88306427e-05],
         [-5.88306427e-05],
         [-5.88306427e-05],
         [-5.88306427e-05],
         [-5.88306427e-05],
         [-5.88306427e-05],
         [-5.88306427e-05],
         [-5.88306427e-05],
         [-5.88306427e-05]], dtype=float64),
  Array([[0.98502685],
         [0.98502685],
         [0.98502685],
         [0.98502685],
         [0.98502685],
         [0.98502685],
         [0.98502685],
         [0.98502685],
         [0.98502685],
         [0.98502685]], dtype=float64)],
 [Array([[-0.06107005,  1.36142286],
         [-0.06107005,  1.36142286],
         [-0.0

In [113]:
result.optimized_trainable_parameters["initial_params"]

[Array([0.0376334 , 2.39761838], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]